# Capstone: Philosophical Factors for NLP
**_Measuring Similarity to Philosophical Concepts in Text Data_**

## Thomas W. Ludlow, Jr.
**General Assembly Data Science Immersive DSI-NY-6**

**February 12, 2019**

# Notebook 5 - Factorizing Unseen Text

### Table of Contents

[**5.1 Prepare Models for Prediction**](#5.1-Prepare-Models-for-Prediction)

[**5.2 Factorizing**](#5.2-Factorizing)
- [5.2.1 Preprocess Unseen Text for Factorizing](#5.2.1-Preprocess-Unseen-Text-for-Factorizing)
- [5.2.2 Factorize Text](#5.2.2-Factorize-Text)
- [5.2.3 Display Results](#5.2.3-Display-Results)

**Libraries**

In [2]:
# Python Data Science
import re
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

# Natural Language Processing
import spacy
import gensim
import pyLDAvis.gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, ldamulticore, CoherenceModel

# Modeling Prep
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

# Neural Net
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

# Override deprecation warnings
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## 5.1 Prepare Models and Input Text

**Load Models**

In [ ]:
logreg = joblib.load('./models/logreg')
mnb = joblib.load('./models/mnb')

In [2]:
rnn = keras.models.load_model('./models/rnn')

ResourceExhaustedError: OOM when allocating tensor with shape[70922,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node training/Adam/Variable_12/Assign (defined at /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:402)  = Assign[T=DT_FLOAT, _grappler_relax_allocator_constraints=true, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/Adam/Variable_12, training/Adam/zeros_12)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training/Adam/Variable_12/Assign', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(cod

**New Text Preprocessing Function**

## 5.2 Factorizing

### 5.2.2 Factorize Text

In [ ]:
def nlp_factorize(lda_df, model=model_full, ss=ss):
    lda_sc = ss.transform(lda_df.values)    
    preds = model.predict_proba(lda_sc)
    return preds

In [ ]:
preds = nlp_factorize(bs_df)

### 5.2.3 Display Results

In [ ]:
def display_nlp_factors(preds, raw_text, doc_number=0, thresh=0.01, target_authors=None):
    print('-'*30)
    print('Doc #:', doc_number)
    if target_authors: print('Target Author:', target_authors[doc_number])
    print('Text:\n', raw_text[doc_number])
    print('\nPhilosophical Factors:\n')
    
    result_list = [(text_data.Author.unique()[i], preds[doc_number][i]) for i in range(len(text_data.Author.unique()))]
    result_list.sort(key=lambda tup: tup[1], reverse=True)
    
    for j in range(len(text_data.Author.unique())):
        n_sp = 35 - len(result_list[j][0])
        if result_list[j][1] >= thresh:
            print('\t{}{}{}'.format(result_list[j][0],str(' '*n_sp),str(round(result_list[j][1], 3))))
    print('-'*30)